In [1]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
import re
from gensim import models

from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, CuDNNLSTM, Conv1D
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")
print("Train shape : ", train_df.shape)
print("Test shape : ", test_df.shape)

Train shape :  (1306122, 3)
Test shape :  (375806, 2)


In [3]:
print(pd.DataFrame.info(test_df))
print(pd.DataFrame.info(train_df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375806 entries, 0 to 375805
Data columns (total 2 columns):
qid              375806 non-null object
question_text    375806 non-null object
dtypes: object(2)
memory usage: 5.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306122 entries, 0 to 1306121
Data columns (total 3 columns):
qid              1306122 non-null object
question_text    1306122 non-null object
target           1306122 non-null int64
dtypes: int64(1), object(2)
memory usage: 29.9+ MB
None


In [4]:
df = pd.concat([train_df.drop('target', axis=1) ,test_df])


In [5]:
df['lowered_question'] = df['question_text'].apply(lambda x: x.lower())

In [6]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [7]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [8]:
df['treated_question'] = df['lowered_question'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [9]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [10]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [11]:
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text



In [12]:
df['treated_question'] = df['treated_question'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [13]:
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

In [14]:
def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x


In [15]:
df['treated_question'] = df['treated_question'].apply(lambda x: correct_spelling(x, mispell_dict))

In [16]:
train_df['question_text'] = df['treated_question'][:1306122]
test_df['question_text'] = df['treated_question'][1306122:]

In [17]:
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=0)

In [18]:
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

## fill up the missing values
train_X = train_df["question_text"].values
val_X = val_df["question_text"].values
test_X = test_df["question_text"].values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

In [19]:
EMBEDDING_FILE = './data/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
embeddings_index = models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = (np.random.rand(nb_words, embed_size) - 0.5) / 5.0
for word, i in word_index.items():
    if i >= max_features: continue
    if word in embeddings_index:
        embedding_vector = embeddings_index.get_vector(word)
        embedding_matrix[i] = embedding_vector

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 1

In [20]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/2
1175509/1175509 [==============================] - 252s 214us/sample - loss: 0.1176 - acc: 0.9545 - val_loss: 0.1025 - val_acc: 0.9597
Epoch 2/2
1175509/1175509 [==============================] - 245s 209us/sample - loss: 0.0942 - acc: 0.9623 - val_loss: 0.1029 - val_acc: 0.9600


In [21]:
pred_google_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_google_val_y>thresh).astype(int))))


130613/130613 [==============================] - 7s 50us/sample
F1 score at threshold 0.1 is 0.6034628566341746
F1 score at threshold 0.11 is 0.6129707112970711
F1 score at threshold 0.12 is 0.6185385718913415
F1 score at threshold 0.13 is 0.6234120636115554
F1 score at threshold 0.14 is 0.6295657175726201
F1 score at threshold 0.15 is 0.6330368858434174
F1 score at threshold 0.16 is 0.6358848141767167
F1 score at threshold 0.17 is 0.6395928956296149
F1 score at threshold 0.18 is 0.6431891673403396
F1 score at threshold 0.19 is 0.6451183280401598
F1 score at threshold 0.2 is 0.648144308521667
F1 score at threshold 0.21 is 0.6500524658971669
F1 score at threshold 0.22 is 0.6518534231319935
F1 score at threshold 0.23 is 0.6536486197303659
F1 score at threshold 0.24 is 0.6550308008213552
F1 score at threshold 0.25 is 0.6574190377368795
F1 score at threshold 0.26 is 0.658876701758254
F1 score at threshold 0.27 is 0.6592135268924858
F1 score at threshold 0.28 is 0.6596723518850988
F1 score 

In [22]:
pred_google_test_y = model.predict([test_X], batch_size=1024, verbose=1)

375806/375806 [==============================] - 19s 50us/sample


In [23]:
del word_index, embeddings_index, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(5)

In [24]:
EMBEDDING_FILE = './data/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17  

In [25]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/2
1175509/1175509 [==============================] - 572s 486us/sample - loss: 0.1159 - acc: 0.9541 - val_loss: 0.1002 - val_acc: 0.9606
Epoch 2/2
1175509/1175509 [==============================] - 247s 210us/sample - loss: 0.0940 - acc: 0.9629 - val_loss: 0.0993 - val_acc: 0.9607


In [26]:
pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))



130613/130613 [==============================] - 7s 51us/sample
F1 score at threshold 0.1 is 0.5812281782706921
F1 score at threshold 0.11 is 0.589227284168135
F1 score at threshold 0.12 is 0.5961300243475289
F1 score at threshold 0.13 is 0.6048050139275766
F1 score at threshold 0.14 is 0.6118741692512184
F1 score at threshold 0.15 is 0.6177807787530948
F1 score at threshold 0.16 is 0.6231559716830327
F1 score at threshold 0.17 is 0.6286720415160781
F1 score at threshold 0.18 is 0.6341944574917803
F1 score at threshold 0.19 is 0.6380420781451266
F1 score at threshold 0.2 is 0.6423075061704496
F1 score at threshold 0.21 is 0.6468799058084773
F1 score at threshold 0.22 is 0.6502580389043271
F1 score at threshold 0.23 is 0.6542328574301227
F1 score at threshold 0.24 is 0.6560593737291582
F1 score at threshold 0.25 is 0.65775126015842
F1 score at threshold 0.26 is 0.6591215865434534
F1 score at threshold 0.27 is 0.661800997637175
F1 score at threshold 0.28 is 0.6631260289978226
F1 score at

In [27]:
pred_glove_test_y = model.predict([test_X], batch_size=1024, verbose=1)

375806/375806 [==============================] - 20s 54us/sample


In [28]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc 
gc.collect()
time.sleep(5)

In [29]:
EMBEDDING_FILE = './data/wiki-news-300d-1M/wiki-news-300d-1M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/2
1175509/1175509 [==============================] - 247s 210us/sample - loss: 0.1151 - acc: 0.9555 - val_loss: 0.1026 - val_acc: 0.9600
Epoch 2/2
1175509/1175509 [==============================] - 247s 210us/sample - loss: 0.0931 - acc: 0.9625 - val_loss: 0.1049 - val_acc: 0.9584


In [31]:
pred_fasttext_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_fasttext_val_y>thresh).astype(int))))    

130613/130613 [==============================] - 7s 51us/sample
F1 score at threshold 0.1 is 0.5341006125858548
F1 score at threshold 0.11 is 0.5431394906879513
F1 score at threshold 0.12 is 0.552614188532556
F1 score at threshold 0.13 is 0.5617058449809402
F1 score at threshold 0.14 is 0.5703520641737228
F1 score at threshold 0.15 is 0.5762069533695832
F1 score at threshold 0.16 is 0.5820526778610611
F1 score at threshold 0.17 is 0.589202077833603
F1 score at threshold 0.18 is 0.5952576339999136
F1 score at threshold 0.19 is 0.6017706872370266
F1 score at threshold 0.2 is 0.6065675892142032
F1 score at threshold 0.21 is 0.6110685129222859
F1 score at threshold 0.22 is 0.6157574649046595
F1 score at threshold 0.23 is 0.6212935507783544
F1 score at threshold 0.24 is 0.6251876876876877
F1 score at threshold 0.25 is 0.6288184711862795
F1 score at threshold 0.26 is 0.6325706594885598
F1 score at threshold 0.27 is 0.6358814656221108
F1 score at threshold 0.28 is 0.6391691686764778
F1 score 

In [32]:
pred_fasttext_test_y = model.predict([test_X], batch_size=1024, verbose=1)

375806/375806 [==============================] - 19s 50us/sample


In [33]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc 
gc.collect()
time.sleep(5)

In [34]:
EMBEDDING_FILE = './data/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/2
1175509/1175509 [==============================] - 246s 209us/sample - loss: 0.1154 - acc: 0.9551 - val_loss: 0.1025 - val_acc: 0.9592
Epoch 2/2
1175509/1175509 [==============================] - 246s 209us/sample - loss: 0.0951 - acc: 0.9625 - val_loss: 0.0995 - val_acc: 0.9608


In [36]:
pred_paragram_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_paragram_val_y>thresh).astype(int))))

130613/130613 [==============================] - 7s 52us/sample
F1 score at threshold 0.1 is 0.600457113286472
F1 score at threshold 0.11 is 0.6073206184204731
F1 score at threshold 0.12 is 0.6140485083022095
F1 score at threshold 0.13 is 0.6207275726198112
F1 score at threshold 0.14 is 0.6265262373007735
F1 score at threshold 0.15 is 0.6333838575217885
F1 score at threshold 0.16 is 0.6389637904468414
F1 score at threshold 0.17 is 0.6421571017602028
F1 score at threshold 0.18 is 0.6474777448071216
F1 score at threshold 0.19 is 0.6496917447746979
F1 score at threshold 0.2 is 0.6531005785040088
F1 score at threshold 0.21 is 0.6547490625160528
F1 score at threshold 0.22 is 0.6580658586068772
F1 score at threshold 0.23 is 0.6606277649041501
F1 score at threshold 0.24 is 0.66322402044293
F1 score at threshold 0.25 is 0.6663438777706048
F1 score at threshold 0.26 is 0.6682990532158015
F1 score at threshold 0.27 is 0.6705895292564893
F1 score at threshold 0.28 is 0.6712207042097078
F1 score a

In [37]:
pred_paragram_test_y = model.predict([test_X], batch_size=1024, verbose=1)

375806/375806 [==============================] - 19s 50us/sample


In [38]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc
gc.collect()
time.sleep(5)

In [39]:
pred_val_y = 0.25*pred_glove_val_y + 0.25*pred_fasttext_val_y + 0.25*pred_paragram_val_y + 0.25*pred_google_val_y
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.5784472957609226
F1 score at threshold 0.11 is 0.5878493418047424
F1 score at threshold 0.12 is 0.5965923539883021
F1 score at threshold 0.13 is 0.6044098573281452
F1 score at threshold 0.14 is 0.6114879746277861
F1 score at threshold 0.15 is 0.6183106804826627
F1 score at threshold 0.16 is 0.6243840116809636
F1 score at threshold 0.17 is 0.6295182116906558
F1 score at threshold 0.18 is 0.6361368663811361
F1 score at threshold 0.19 is 0.640974212034384
F1 score at threshold 0.2 is 0.6450768410335967
F1 score at threshold 0.21 is 0.648640690941211
F1 score at threshold 0.22 is 0.6521587916728772
F1 score at threshold 0.23 is 0.6551915043535155
F1 score at threshold 0.24 is 0.6587851610273135
F1 score at threshold 0.25 is 0.6623390005151983
F1 score at threshold 0.26 is 0.6647552520460825
F1 score at threshold 0.27 is 0.6676157334176948
F1 score at threshold 0.28 is 0.6696842217370467
F1 score at threshold 0.29 is 0.6724547023295945
F1 score at threshold 0.

In [40]:
pred_test_y = 0.25*pred_glove_test_y + 0.25*pred_fasttext_test_y + 0.25*pred_paragram_test_y + 0.25*pred_google_test_y
pred_test_y = (pred_test_y>0.34).astype(int)
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)